In [1]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import matplotlib.pyplot as plt
import concurrent.futures
import pickle
import warnings
from datetime import date
import hisepy
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor
from tqdm import tqdm
import anndata
import gc
warnings.filterwarnings("ignore")
sc.settings.n_jobs = 60
print("Current working directory:", os.getcwd())

Current working directory: /home/jupyter/BRI_Figures_Final_V2/Figure2/02_DEG


# Read MetaData

In [3]:
meta_data=pd.read_csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA_meta_data-2024-05-09.csv")

# Assemble Data

In [4]:
meta_data_subset=meta_data[meta_data['sample.visitName'].isin(['Flu Year 1 Day 0'])]

In [5]:
meta_data_subset=meta_data_subset[meta_data_subset['Covid_exclusion']=='no']

In [6]:
%%time
file_names= ['/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/h5ad/sample_h5ad/'+x+".h5ad" for x in meta_data_subset['pbmc_sample_id'].tolist()]
adata_list = []
with ThreadPoolExecutor(max_workers=30) as executor:
    future_to_file = {executor.submit(sc.read_h5ad, file_name): file_name for file_name in file_names}
    for future in tqdm(as_completed(future_to_file), total=len(file_names)):
        result = future.result()
        if result is not None:
            adata_list.append(result)

100% 92/92 [02:29<00:00,  1.62s/it] 

CPU times: user 16.2 s, sys: 21.4 s, total: 37.7 s
Wall time: 2min 30s


In [7]:
adata = anndata.concat(adata_list)
del adata_list 
gc.collect()

0

# Read Files and filter genes for DESEQ2

In [8]:
fitlered_gene=pd.DataFrame()
for i in adata.obs['AIFI_L3'].unique():
    print(i)
    adata_subset=adata[adata.obs['AIFI_L3']==i]
    sc.pp.filter_genes(adata_subset, min_cells=round(np.shape(adata_subset.X)[0]*0.1))
    gene_list=pd.DataFrame(list(adata_subset.var.index))
    gene_list.columns=['gene']
    gene_list['AIFI_L3']=i
    fitlered_gene=pd.concat([fitlered_gene,gene_list])

GZMB- CD27+ EM CD4 T cell
ISG+ CD14 monocyte
GZMK+ Vd2 gdT
GZMB- CD27- EM CD4 T cell
CD56bright NK cell
Early memory B cell
Core naive B cell
CLP cell
ISG+ memory CD8 T cell
IL1B+ CD14 monocyte
Proliferating T cell
Activated memory B cell
KLRF1+ GZMB+ CD27- EM CD8 T cell
C1Q+ CD16 monocyte
ISG+ naive CD4 T cell
ASDC
CM CD4 T cell
CD8aa
Intermediate monocyte
Type 2 polarized memory B cell
ISG+ CD16 monocyte
Core naive CD8 T cell
GZMK+ CD56dim NK cell
KLRF1+ effector Vd1 gdT
Core CD14 monocyte
ISG+ naive B cell
Naive Vd1 gdT
Transitional B cell
Core naive CD4 T cell
ISG+ MAIT
CM CD8 T cell
HLA-DRhi cDC2
Core memory B cell
ISG+ CD56dim NK cell
pDC
GZMK+ CD27+ EM CD8 T cell
Core CD16 monocyte
CD95 memory B cell
cDC1
Memory CD8 Treg
GZMK- CD56dim NK cell
Platelet
CD27+ effector B cell
GZMB+ Vd2 gdT
ISG+ cDC2
Naive CD4 Treg
SOX4+ naive CD8 T cell
Adaptive NK cell
Memory CD4 Treg
GZMK- CD27+ EM CD8 T cell
CD14+ cDC2
SOX4+ naive CD4 T cell
ILC
Plasma cell
KLRF1- GZMB+ CD27- EM CD8 T cell
Proli

In [9]:
fitlered_gene.to_csv("filtered_gene_Y1D0.csv")